CS594 Green Line
[Due on Oct19]Step1: Collect data on list of dispensaries from online : 
1. In Phase1, fields are as below:
>name
>street lnie 1,street line 2,city,state,zip code
>tel

2. As for weeds lists, we assume that the dispensaries have their own inventories like a CSV or .xls file containing the usage(recreational, medical, etc), storage(0-10^10 gram), unit prices for each weed items.

3. Pase2:
>image(TBD), open time


In [1]:
import io, time, json
import requests
from bs4 import BeautifulSoup
import re

In [2]:
def parse_page(html):
    """
    Parse the dispensary links on a single page of a websites.
    
    Args:
        html (string): String of HTML

    Returns:
        
    """
    soup = BeautifulSoup(html,'html.parser')
    urls = soup.find_all('a', itemprop="url")
    #use beautifulsoup CSS selectors to select and construct the review dict
    d_links=[]
    for url in urls:
        d_link = url.get('href')
        d_links.append(d_link)
    return d_links

In [3]:
def retrieve_html(url, parse_f=parse_page):
    """
    Return the raw HTML at the specified URL.
    """
    resp = requests.request('GET', url)
    print(resp)
    dispensary_links = parse_f(resp.text)
    print(dispensary_links)
    #use a list to store all dispensary links
    
    return resp.status_code, dispensary_links

In [4]:
#retrieve_html('https://wheresweed.com/illinois/chicago/marijuana-dispensaries')

In [77]:
#https://potguide.com/illinois/marijuana-dispensaries/chicago/
def parse_Chicago_dispensaries_page(html):
    """
    Parse the reviews on a single page of a restaurant.
    
    Args:
        html (string): String of HTML corresponding to a Yelp restaurant

    Returns:
        tuple(list, string): a tuple of two elements
            first element: list of dictionaries corresponding to the extracted review information
            second element: URL for the next page of reviews (or None if it is the last page)
    """
    soup = BeautifulSoup(html,'html.parser')

    divs = soup.find_all('div', "basic-listing")
    
    #use beautifulsoup CSS selectors to select and construct the review dict
    dispensaries = []
    for d in divs:
        dispensary={}
#         d_profile = "https://potguide.com/" + d.contents[1].contents[0].get('href')
        d_name = d.contents[1].contents[0].string
        d_add = d.contents[2].string.strip() + ", " + d.contents[4].string.strip()
        d_phone = d.contents[6].string.strip().lstrip(" Phone: ")
        d_website = d.contents[11].get("href")
        
        dispensary["fullname"] = d_name
#         dispensary["profile"] = d_profile
        dispensary["add"] = d_add
        dispensary["phone"] = d_phone
        dispensary["website"] = d_website
        dotComName = d_website.split('/')[2].split('.')
        # design1: shortname will be seen as one attribute in menu
        # design2: the menu of weeds will be one attribute in dispensary #This seems better when design the whole website flow
        if len(dotComName) == 2: 
            dispensary["shortname"] = dotComName[0]
        else:
            dispensary["shortname"] = dotComName[1]
#         print(d.contents)
        dispensary["menu"] = None # init it as None, later on we will update it after we got the data
        dispensaries.append(dispensary)
        print("d_website", d_website)
    return dispensaries

# Stage 2 Some other webiste to collect dispensaries info

## 1 Work on new dispensary Json
TBD

In [78]:
# name, category, flavor, unit price, totals-in-storage
stat_code, dispensaries2 = retrieve_html("https://potguide.com/illinois/marijuana-dispensaries/chicago/", 
                              parse_Chicago_dispensaries_page)

<Response [200]>
d_website https://col-care.com/location/chicago?utm_source=PotGuide&sid=159225
d_website http://dispensary33.com/?utm_source=PotGuide&sid=159226
d_website http://greengatechicago.com/?utm_source=PotGuide&sid=499029
d_website https://www.maribisllc.com/?utm_source=PotGuide&sid=159240
d_website https://www.medmardispensary.com/?utm_source=PotGuide&sid=159242
d_website http://www.midwaydispensary.com/?utm_source=PotGuide&sid=159244
d_website https://missiondispensaries.com/chicago-il/?utm_source=PotGuide&sid=179810
d_website https://moderncann.com/?utm_source=PotGuide&sid=159247
d_website http://www.numed.com/chicago?utm_source=PotGuide&sid=584880
d_website http://theherbalcarecenter.com/?utm_source=PotGuide&sid=159266
d_website http://www.zenleafil.com/?utm_source=PotGuide&sid=159270
[{'fullname': 'Columbia Care - Chicago', 'add': '4758 N Milwaukee Ave, Chicago, IL 60630', 'phone': '(312) 948-9082', 'website': 'https://col-care.com/location/chicago?utm_source=PotGuide&si

In [79]:
with open('dispensary_list_at_Chicago.json', 'w') as fp:
    json.dump(dispensaries2, fp)

## 2 Work on the weed list in each dispensary
After working on the webpage sources, only some dispensaries are allowed to crawled its menu data. They are dispensary33.com, greengatechicago.com, numed.com, moderncann.com

>1 numed is downloaded/copied directly from its page source
>
>2 moca is http request forbidden. But we can manually copy-paste the xml menu content from its webpage resource. ( then parse and convert xml to json)
>
>structure is as below:
id: the id of weed item<br>
name: the name of weed item<br>
product_type: based on its appearance, it's divided into 'Edible', 'DryFlower', 'Pre-RolledJoints', 'Concentrates'<br>
strain_type: 'indica', 'sativa', 'hybrid'<br>
flavor: ['sweet', 'peech','apple', 'strawberry']<br>
effect: {"positive": null or ["Relaxed", ...],"negative": null or ["Dizzy"], "medical": null or ["Depression", "Insomnia", ...]}<br>
price: 60, 30, 45, etc <br> # assume a uniform unit per gram, otherwise hard to find the cheapest price among dispensaries<br>
quantity: the # of weeds from per dispensary in storage, 0 - 99999, assume its unit is kilogram
description: string


```json
{
    "id": 649090341,
    "name": "1:1 Balance Spray Tincture",
    "product_type": "Edible",
    "vendor": "Cresco",
    "thc": "200.6",
    "cbd": "100.71",
    "strain_type": null,
    "flavor": null or ["sweet", "strawberry"]
    "effect": {"positive": null or ["Relaxed", ...],
                "negative": null or ["Dizzy"],
                "medical": null or ["Depression", "Insomnia", ...]
                }
    "price": "30.0",
    "quantity": 999,
    "description": string value
  }
```

In [16]:
#https://moderncann.com/menu/
#1. manually copy down the <input> from the source page, then
#2. use below function to extract the info for different product
def parse_moca_menu(filename):
    """
    Parse the menu on a single page of moderncannabis.com
    
    Args:
        filename (string): filename for a file storing string of xml corresponding to moca product menu

    Returns:
        tuple(list, string): a tuple of two elements
            first element: list of dictionaries corresponding to the extracted review information
            second element: URL for the next page of reviews (or None if it is the last page)
    """
    with open('menus/moca_menu.xml') as fd:
        soup = BeautifulSoup(fd, 'html.parser')
    
    inputs = soup.find_all('input')
    print("len(inputs)", len(inputs))
    #use beautifulsoup CSS selectors to select and construct the review dict
    products = []
    # every 16 items, we generate a new product
    for i in range(int(len(inputs)/16)):
        product={}
        product["id"] = inputs[i*16].get("id").split("_")[1]
        product["name"] = inputs[i*16+1].get("value")
        product["product_type"] = inputs[i*16].get("value")
        product["vendor"] = inputs[i*16+8].get("value")
        product["thc"] = inputs[i*16+3].get("value")
        product["cbd"] = inputs[i*16+6].get("value")
        product["strain_type"] = inputs[i*16+10].get("value")
        ### To be inserted from data.json
        product["flavor"] = None
        product["effect"] = {}
        product["effect"]["positive"] = None
        product["effect"]["negative"] = None
        product["effect"]["medical"] = None
        ### To be inserted from data.json
        product_weight = float(inputs[i*16+9].get("value"))
        #print("product_weight", product_weight)
        prodcut_price = round(float(inputs[i*16+15].get("value")) / product_weight, 2) if product_weight != 0 else inputs[i*16+15].get("value")
        #print("prodcut_price", prodcut_price)
        product["price"] = str(prodcut_price) 
        product["quantity"] = 0 if inputs[i*16+15].get("value") == "0" else 999
        product["description"] = inputs[i*16+13].get("value")
        products.append(product)
        #print("product", product)
    return products

In [13]:
# stat_code, products = retrieve_html("https://moderncann.com/menu/", parse_moca_page)
retrieve_html("https://moderncann.com/menu/", parse_moca_menu)

<Response [403]>
len(inputs) 4688
product_weight 0.0
prodcut_price 15
product {'id': '4821', 'name': 'ASH CATCHER', 'product_type': 'ASH CATCHERS', 'vendor': 'HSW', 'thc': '0.00', 'cbd': '0.00', 'strain_type': '', 'flavor': None, 'effect': {'positive': None, 'negative': None, 'medical': None}, 'price': '15', 'quantity': 999, 'description': ''}
product_weight 0.0
prodcut_price 7
product {'id': '3986', 'name': 'YOCAN STIX ATOMIZER', 'product_type': 'ATOMIZERS', 'vendor': 'GOT VA', 'thc': '0.00', 'cbd': '0.00', 'strain_type': '', 'flavor': None, 'effect': {'positive': None, 'negative': None, 'medical': None}, 'price': '7', 'quantity': 999, 'description': ''}
product_weight 0.0
prodcut_price 45
product {'id': '4609', 'name': 'PUFFCO PEAK ATOMIZER', 'product_type': 'ATOMIZERS', 'vendor': 'HSW', 'thc': '0.00', 'cbd': '0.00', 'strain_type': '', 'flavor': None, 'effect': {'positive': None, 'negative': None, 'medical': None}, 'price': '45', 'quantity': 999, 'description': ''}
product_weight 1.0

prodcut_price 14.29
product {'id': '5080', 'name': 'BLUE SHARK', 'product_type': 'FLOWERS', 'vendor': 'SHELBY', 'thc': '5.33', 'cbd': '8.48', 'strain_type': '', 'flavor': None, 'effect': {'positive': None, 'negative': None, 'medical': None}, 'price': '14.29', 'quantity': 999, 'description': ''}
product_weight 1.0
prodcut_price 50.0
product {'id': '4801', 'name': 'BLUEBERRY X CLEMENTINE', 'product_type': 'HASH', 'vendor': 'REVOLU', 'thc': '56.25', 'cbd': '0.00', 'strain_type': '', 'flavor': None, 'effect': {'positive': None, 'negative': None, 'medical': None}, 'price': '50.0', 'quantity': 999, 'description': ''}
product_weight 1.0
prodcut_price 50.0
product {'id': '4802', 'name': 'COOKIES AND CREAM', 'product_type': 'HASH', 'vendor': 'REVOLU', 'thc': '52.76', 'cbd': '0.11', 'strain_type': '', 'flavor': None, 'effect': {'positive': None, 'negative': None, 'medical': None}, 'price': '50.0', 'quantity': 999, 'description': ''}
product_weight 1.0
prodcut_price 50.0
product {'id': '5049', 'n

(403,
 [{'id': '4821',
   'name': 'ASH CATCHER',
   'product_type': 'ASH CATCHERS',
   'vendor': 'HSW',
   'thc': '0.00',
   'cbd': '0.00',
   'strain_type': '',
   'flavor': None,
   'effect': {'positive': None, 'negative': None, 'medical': None},
   'price': '15',
   'quantity': 999,
   'description': ''},
  {'id': '3986',
   'name': 'YOCAN STIX ATOMIZER',
   'product_type': 'ATOMIZERS',
   'vendor': 'GOT VA',
   'thc': '0.00',
   'cbd': '0.00',
   'strain_type': '',
   'flavor': None,
   'effect': {'positive': None, 'negative': None, 'medical': None},
   'price': '7',
   'quantity': 999,
   'description': ''},
  {'id': '4609',
   'name': 'PUFFCO PEAK ATOMIZER',
   'product_type': 'ATOMIZERS',
   'vendor': 'HSW',
   'thc': '0.00',
   'cbd': '0.00',
   'strain_type': '',
   'flavor': None,
   'effect': {'positive': None, 'negative': None, 'medical': None},
   'price': '45',
   'quantity': 999,
   'description': ''},
  {'id': '1996',
   'name': 'CHOCOLATE CHIP COOKIE 1:1 4 PACK',
   'p

In [17]:
moca_products = parse_moca_menu("menus/moca_menu.xml")
with open('menus/menu_moderncann.json', 'w') as fp:
    json.dump(moca_products, fp)

len(inputs) 4688


#### 2.1 other dispensaries
>For midway Dispensary, it has a online pdf menu
http://www.midwaydispensary.com/?utm_source=PotGuide&sid=159244

https://midwaydispensary.com/wp-content/uploads/sites/4805/2019/11/ONLINE-STORE-MENU-11-6.pdf

>https://www.maribisllc.com/?utm_source=PotGuide&sid=159240 in-store only

In [20]:
url33 = "https://dispensary33.com/products/"
resp33 = requests.request('GET', url33)

In [21]:
#https://potguide.com/illinois/marijuana-dispensaries/chicago/
def extract_dispensary33_menu(html):
    """
    
    """
    soup = BeautifulSoup(html,'html.parser')

    divs = soup.find_all('div', "type_container")
    print("len(divs)", len(divs))
    #use beautifulsoup CSS selectors to select and construct the review dict
    products = []
#     for soupd in divs[:1]:
    for soupd in divs:
        subdivs = soupd.find_all('div', "productThumbPopup")
        print("len(subdivs)", len(subdivs))
#         for d in subdivs[:1]:
        for d in subdivs:
            # new version
            product={}
            product["id"] = d.get("data-id")
            product["name"] = d.contents[3].contents[3].string
            product["product_type"] = soupd.contents[1].h2.string
            product["vendor"] = d.contents[3].contents[1].string
            product["thc"] = d.contents[5].contents[1].contents[0].contents[1].string.rstrip('%')
            product["cbd"] = d.contents[5].contents[1].contents[1].contents[1].string.rstrip('%')
            product["strain_type"] = soupd.contents[1].h2.string
            ### To be inserted from data.json
            product["flavor"] = None
            product["effect"] = {}
            product["effect"]["positive"] = None
            product["effect"]["negative"] = None
            product["effect"]["medical"] = None
            ### To be inserted from data.json
#             product_weight = float(inputs[i*16+9].get("value"))
#             print("product_weight", product_weight)
#             prodcut_price = round(float(inputs[i*16+15].get("value")) / product_weight, 2) if product_weight != 0 else inputs[i*16+15].get("value")
#             print("prodcut_price", prodcut_price)
            product["price"] = 45 # fake data 
            product["quantity"] = 999 # fake data 
            product["description"] = d.contents[5].contents[5].string
            products.append(product)
            print("product", product)
    return products

In [22]:
products33 = extract_dispensary33_menu(resp33.text)

len(divs) 7
len(subdivs) 10
product {'id': '29110', 'name': 'MAC 1', 'product_type': 'Dry Flower', 'vendor': 'Aeriz', 'thc': '25.69', 'cbd': '0.00', 'strain_type': 'Dry Flower', 'flavor': None, 'effect': {'positive': None, 'negative': None, 'medical': None}, 'price': 45, 'quantity': 999, 'description': 'Originally bred by the legendary Capulator, we believe this strain to be a backcross of the incredibly popular MAC (Alien Cookies x (Colombian Gold x Starfighter)) and is lauded in grower circles. Well known for its heavy trichome coverage and incredibly dense buds, this example holds true to its name. A heady and nuanced cerebral euphoria with a relaxing physical quality. Notes of baking spice and citrus, with a sweet, yet floral backdrop.'}
product {'id': '6015', 'name': 'Cherry Wu', 'product_type': 'Dry Flower', 'vendor': 'Bedford Grow', 'thc': '0.78', 'cbd': '17.15', 'strain_type': 'Dry Flower', 'flavor': None, 'effect': {'positive': None, 'negative': None, 'medical': None}, 'price'

In [23]:
with open('menus/menu_dispensary33.json', 'w') as fp:
    json.dump(products33, fp)

#### numed dispensary

In [26]:
with open('menus/menu_numed_old.json', 'r') as fp:
    numed_weeds = json.load(fp)
for weed in numed_weeds[:2]:
    print(weed["name"])

1:1 Balance Spray Tincture
1:1 Hard Sweet Cherry


## 3 We will build our own format of json before storing it in MongoDB